# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115095e+02     1.687663e+00
 * time: 0.6361551284790039
     1     1.103933e+01     9.430105e-01
 * time: 1.5641651153564453
     2    -1.115763e+01     9.598823e-01
 * time: 1.6546151638031006
     3    -3.379430e+01     7.822546e-01
 * time: 1.7856462001800537
     4    -4.741303e+01     6.078616e-01
 * time: 1.9219801425933838
     5    -5.672582e+01     2.115185e-01
 * time: 2.0290331840515137
     6    -5.976286e+01     1.280326e-01
 * time: 2.110234022140503
     7    -6.090683e+01     5.413073e-02
 * time: 2.209315061569214
     8    -6.131510e+01     8.291316e-02
 * time: 2.2947351932525635
     9    -6.163287e+01     3.273364e-02
 * time: 2.3764171600341797
    10    -6.183792e+01     2.220246e-02
 * time: 2.456792116165161
    11    -6.199845e+01     1.702297e-02
 * time: 2.5556840896606445
    12    -6.206155e+01     1.680790e-02
 * time: 2.63677716255188
    13    -6.212810e+01     1.272768e-02
 * time: 2.720700025558471

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671068
    AtomicLocal         -18.8557712
    AtomicNonlocal      14.8522675
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485378 
    Xc                  -19.3336822

    total               -62.261666459900